Чтобы использовать датасет, подготовленный для YOLO, для обучения keras-ocr, вам нужно преобразовать данные в формат, который понимает keras-ocr.

1. Разница между форматами YOLO и Keras-OCR
YOLO формат обычно хранит аннотации в .txt-файлах с координатами bounding box в формате:

Copy
class_id x_center y_center width height
(нормализованные относительно ширины и высоты изображения).

Keras-OCR ожидает данные в виде списка пар (image, [текст1, текст2, ...]), где bounding boxes могут быть в формате [[x1, y1], [x2, y2], [x3, y3], [x4, y4]] (координаты углов полигона).

2. Преобразование YOLO → Keras-OCR

# Шаг 1. Загрузка и парсинг YOLO-аннотаций

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import keras_ocr
from tensorflow.keras.optimizers import Adam


2025-04-22 22:17:29.194712: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 22:17:29.985474: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-22 22:17:29.985498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-22 22:17:30.030712: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 22:17:30.139236: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def parse_yolo_annotation(img_path, label_path):
    # Загружаем изображение, чтобы узнать его размеры
    img = cv2.imread(img_path)
    img_h, img_w = img.shape[:2]
    
    # Читаем аннотации из .txt-файла
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    boxes = []
    for line in lines:
        class_id, x_center, y_center, w, h = map(float, line.strip().split())
        
        # Конвертация из нормализованных координат YOLO в абсолютные
        x_center *= img_w
        y_center *= img_h
        w *= img_w
        h *= img_h
        
        # Вычисление координат углов прямоугольника (x1, y1, x2, y2)
        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)
        
        # Добавляем в формате полигона (4 угла)
        box = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
        boxes.append(box)
    
    return img, boxes

# Шаг 2. Создание датасета для Keras-OCR

Keras-OCR ожидает данные в виде списка пар (изображение, список текстов). Если ваш датасет содержит только цифры, можно сгенерировать метки автоматически (предполагая, что class_id соответствует цифре):

In [3]:
def load_yolo_dataset(data_dir):
    dataset = []
    for img_file in os.listdir(os.path.join(data_dir, "images")):
        if not img_file.endswith(('.jpg', '.png')):
            continue
        
        img_path = os.path.join(data_dir, "images", img_file)
        label_path = os.path.join(data_dir, "labels", img_file.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        if not os.path.exists(label_path):
            continue
        
        img, boxes = parse_yolo_annotation(img_path, label_path)
        
        # Предполагаем, что class_id = цифра (0 → "0", 1 → "1" и т. д.)
        texts = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id = int(line.strip().split()[0])
                texts.append(str(class_id))
        
        dataset.append((img, texts))  # Keras-OCR ожидает (image, [text1, text2, ...])
    
    return dataset

# Шаг 3. Обучение Keras-OCR

In [ ]:
# Загрузка датасета
dataset = load_yolo_dataset("/home/lastinm/PROJECTS/DATA/syntetic digits")

train_dataset = dataset[:int(0.8 * len(dataset))]
val_dataset = dataset[int(0.8 * len(dataset)):]

In [5]:
def build_keras_ocr_compatible_dataset(yolo_dataset):
    """Преобразует YOLO-датасет в формат, совместимый с keras-ocr"""
    images = []
    texts = []
    boxes = []
    
    for img, annotations in yolo_dataset:
        # Преобразование изображения
        if len(img.shape) == 2:  # Чёрно-белое
            img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:  # Цветное
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_normalized = img_rgb.astype(np.float32) / 255.0
        
        # Обработка аннотаций
        line_texts = []
        line_boxes = []
        
        if isinstance(annotations, str):  # Если аннотация - строка (класс)
            line_texts.append(annotations)
            line_boxes.append([[0, 0], [img.shape[1], 0], 
                             [img.shape[1], img.shape[0]], [0, img.shape[0]]])
        elif isinstance(annotations, list):
            for ann in annotations:
                if isinstance(ann, str):  # Просто класс/текст
                    line_texts.append(ann)
                    line_boxes.append([[0, 0], [img.shape[1], 0], 
                                     [img.shape[1], img.shape[0]], [0, img.shape[0]]])
                elif len(ann) >= 2:  # Кортеж/список (текст, bbox)
                    line_texts.append(str(ann[0]))
                    line_boxes.append(ann[1])
                else:  # Неизвестный формат
                    print(f"Пропущена аннотация неизвестного формата: {ann}")
        
        images.append(img_normalized)
        texts.append(line_texts)
        boxes.append(line_boxes)
    
    return images, texts, boxes

In [6]:
print("Первый элемент train_dataset:", train_dataset[0])
print("Тип аннотаций:", type(train_dataset[0][1]))
print("Содержимое аннотаций:", train_dataset[0][1])

Первый элемент train_dataset: (array([[[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       ...,

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215,

In [7]:
train_images, train_texts, train_boxes = build_keras_ocr_compatible_dataset(train_dataset)
print("Пример преобразованных данных:")
print("Текст:", train_texts[0])
print("Боксы:", train_boxes[0])

Пример преобразованных данных:
Текст: ['6', '0', '4']
Боксы: [[[0, 0], [640, 0], [640, 640], [0, 640]], [[0, 0], [640, 0], [640, 640], [0, 640]], [[0, 0], [640, 0], [640, 640], [0, 640]]]


In [15]:
# Создание генераторов
batch_size = 16
train_gen = data_generator(train_dataset, batch_size)
val_gen = data_generator(val_dataset, batch_size)

# Обучение модели
history = pipeline.recognizer.model.fit(
    train_gen,
    steps_per_epoch=len(train_dataset) // batch_size,
    validation_data=val_gen,
    validation_steps=len(val_dataset) // batch_size,
    epochs=10
)

Epoch 1/10


ValueError: in user code:

    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.w